In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


## Initialize a nvcc plugin for python notebook

In [ ]:
!pip install nvcc4jupyter

## Load the plugin extension

In [ ]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpktvosvsg".


## Retrieve some info about the CUDA device

In [ ]:
%%cuda
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <cuda.h>

int main(int argc, char const *argv[])
{
    // retrieve some info about the CUDA device
    int nDevices;
    cudaGetDeviceCount(&nDevices);
    for (int i = 0; i < nDevices; i++) {
      cudaDeviceProp prop;
      cudaGetDeviceProperties(&prop, i);
      printf("Device Number: %d\n", i);
      printf("  Device name: %s\n", prop.name);
      printf("  max Blocks Per MultiProcessor: %d\n", prop.maxBlocksPerMultiProcessor);
      printf("  max Threads Per MultiProcessor: %d\n", prop.maxThreadsPerMultiProcessor);
      printf("  max Threads Per Block: %d\n", prop.maxThreadsPerBlock);
      printf("  num SM: %d\n", prop.multiProcessorCount);
      printf("  num bytes sharedMem Per Block: %d\n", prop.sharedMemPerBlock);
      printf("  num bytes sharedMem Per Multiprocessor: %d\n", prop.sharedMemPerMultiprocessor);
      printf("  Memory Clock Rate (KHz): %d\n",
           prop.memoryClockRate);
      printf("  Memory Bus Width (bits): %d\n",
           prop.memoryBusWidth);
      printf("  Peak Memory Bandwidth (GB/s): %f\n\n",
           2.0*prop.memoryClockRate*(prop.memoryBusWidth/8)/1.0e6);
    }
    return 0;
}

Device Number: 0
  Device name: Tesla T4
  max Blocks Per MultiProcessor: 16
  max Threads Per MultiProcessor: 1024
  max Threads Per Block: 1024
  num SM: 40
  num bytes sharedMem Per Block: 49152
  num bytes sharedMem Per Multiprocessor: 65536
  Memory Clock Rate (KHz): 5001000
  Memory Bus Width (bits): 256
  Peak Memory Bandwidth (GB/s): 320.064000




## Tile = 32

In [ ]:
%%cuda

#include <cuda.h>
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <math.h>

#define MASK_SIZE 5
#define FILTER_RADIUS (MASK_SIZE / 2)

// tile size
#define IN_TILE_DIM 32
#define OUT_TILE_DIM (IN_TILE_DIM - 2 * FILTER_RADIUS)

__constant__ float dc_F[(2 * FILTER_RADIUS + 1) * (2 * FILTER_RADIUS + 1)];


// Performing 2D convolution on the CPU to verify the correctness of the GPU results.
void host_convolution_2d(float *N, float *F, float *P, int width, int height, int r)
{
    int filter_dim = 2 * r + 1;
    for (int row = 0; row < height; row++)
    {
        for (int col = 0; col < width; col++)
        {
            float p_value = 0.0f;
            for (int f_row = 0; f_row < filter_dim; f_row++)
            {
                for (int f_col = 0; f_col < filter_dim; f_col++)
                {
                    int n_row = row - r + f_row;
                    int n_col = col - r + f_col;
                    if (n_row >= 0 && n_row < height && n_col >= 0 && n_col < width)
                    {
                        p_value += N[n_row * width + n_col] * F[f_row * filter_dim + f_col];
                    }
                }
            }
            P[row * width + col] = p_value;
        }
    }
}

// Verify the difference between the GPU convolution result and the CPU result.
void verify_convolution_result(float *N, float *F, float *P_gpu, int width, int height, int r, float threshold = 1e0f)
{
    // Allocate CPU output array
    float *P_cpu = (float *)malloc(width * height * sizeof(float));

    // Using CPU to perform convolution
    host_convolution_2d(N, F, P_cpu, width, height, r);

    // Calculation error
    double max_error = 0.0;
    double sum_error = 0.0;
    for (int i = 0; i < width * height; i++)
    {
        double err = fabs(P_cpu[i] - P_gpu[i]);
        if (err > max_error)
        {
            max_error = err;
        }
        sum_error += err;
    }

    double avg_error = sum_error / (width * height);

    printf("Verification Results:\n");
    printf("Max error: %e\n", max_error);
    printf("Average error: %e\n", avg_error);

    if (max_error < threshold)
    {
        printf("Result is correct within the threshold.\n");
    }
    else
    {
        printf("Result differs more than the threshold!\n");
    }

    // Release memory
    free(P_cpu);
}


// basic 2d convolution, each thread compute the each element of the output tensor
__global__ void convolution_2d_basic_kernel(float *N, float *F, float *P, int r, int width, int height)
{
    /*
    N: input tensor
    F: filter tensor
    P: output tensor
    r: filter radius (2r+1)x(2r+1)
    width: width of input tensor
    height: height of input tensor
    N and P has the same size
    */

    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int row = blockIdx.y * blockDim.y + threadIdx.y;

    if (col < width && row < height) {
        float p_value = 0.0;
        for (int f_row = 0; f_row < 2 * r + 1; f_row++)
        {
            for (int f_col = 0; f_col < 2 * r + 1; f_col++)
            {
                int n_row = row - r + f_row;
                int n_col = col - r + f_col;
                if (n_row >= 0 && n_row < height && n_col >= 0 && n_col < width)
                {
                    p_value += N[n_row * width + n_col] * F[f_row * (2 * r + 1) + f_col];
                }
            }
        }
        P[row * width + col] = p_value;
    }
}

// constatn memory version of 2d convolution, each thread compute the each element of the output tensor
__global__ void convolution_2d_constant_mem_kernel(float *N, float *P, int r, int width, int height)
{
    /*
    N: input tensor
    P: output tensor
    r: filter radius (2r+1)x(2r+1)
    width: width of input tensor
    height: height of input tensor
    N and P has the same size
    use constant memory for the filter tensor F
    */

    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int row = blockIdx.y * blockDim.y + threadIdx.y;

    if (col < width && row < height) {
        float p_value = 0.0;
        for (int f_row = 0; f_row < 2 * r + 1; f_row++)
        {
            for (int f_col = 0; f_col < 2 * r + 1; f_col++)
            {
                int n_row = row - r + f_row;
                int n_col = col - r + f_col;
                if (n_row >= 0 && n_row < height && n_col >= 0 && n_col < width)
                {
                    p_value += N[n_row * width + n_col] * dc_F[f_row * (2 * r + 1) + f_col];
                }
            }
        }
        P[row * width + col] = p_value;
    }
}

// tiled version of 2d convolution, a block of threads compute a tile of the output tensor, use the shared memory to cache the input tensor
__global__ void convolution_2d_tiled_const_mem_kernel(float *N, float *P, int width, int height)
{
    /*
    N: input tensor
    P: output tensor
    width: width of input tensor
    height: height of input tensor
    N and P has the same size
    use constant memory for the filter tensor F
    */

    __shared__ float s_N[IN_TILE_DIM][IN_TILE_DIM];

    int col = blockIdx.x * OUT_TILE_DIM + threadIdx.x - FILTER_RADIUS;
    int row = blockIdx.y * OUT_TILE_DIM + threadIdx.y - FILTER_RADIUS;

    if (col >= 0 && col < width && row >= 0 && row < height)
    {
        s_N[threadIdx.y][threadIdx.x] = N[row * width + col];
    }
    else
    {
        s_N[threadIdx.y][threadIdx.x] = 0.0;
    }

    __syncthreads();

    int tile_col = threadIdx.x - FILTER_RADIUS;
    int tile_row = threadIdx.y - FILTER_RADIUS;

    if (col >= 0 && col < width && row >= 0 && row < height)
    {
        if (tile_col >= 0 && tile_col < OUT_TILE_DIM && tile_row >= 0 && tile_row < OUT_TILE_DIM)
        {
            float p_value = 0.0;
            for (int f_row = 0; f_row < 2 * FILTER_RADIUS + 1; f_row++)
            {
                for (int f_col = 0; f_col < 2 * FILTER_RADIUS + 1; f_col++)
                {
                    p_value += s_N[tile_row + f_row][tile_col + f_col] * dc_F[f_row * (2 * FILTER_RADIUS + 1) + f_col];
                }
            }
            P[row * width + col] = p_value;
        }
    }
}

#define TILE_DIM 32
// tiled version of 2d convolution, a block of threads compute a tile of the output tensor, use the shared memory to cache the input tensor
__global__ void convolution_2d_cached_tiled_const_mem_kernel(float *N, float *P,
                                                             int width,
                                                             int height)
{
    /*
    N: input tensor
    P: output tensor
    width: width of input tensor
    height: height of input tensor
    N and P has the same size
    use constant memory for the filter tensor F
    */

    __shared__ float s_N[TILE_DIM][TILE_DIM];

    int col = blockIdx.x * TILE_DIM + threadIdx.x;
    int row = blockIdx.y * TILE_DIM + threadIdx.y;
    if (col >= 0 && col < width && row >= 0 && row < height)
    {
        s_N[threadIdx.y][threadIdx.x] = N[row * width + col];
    }
    else
    {
        s_N[threadIdx.y][threadIdx.x] = 0.0;
    }

    __syncthreads();

    if (col < width && row < height)
    {
        float p_value = 0.0;

        for (int f_row = 0; f_row < 2 * FILTER_RADIUS + 1; f_row++)
        {
            for (int f_col = 0; f_col < 2 * FILTER_RADIUS + 1; f_col++)
            {
                if (threadIdx.x - FILTER_RADIUS + f_col >= 0 &&
                    threadIdx.x - FILTER_RADIUS + f_col < TILE_DIM &&
                    threadIdx.y - FILTER_RADIUS + f_row >= 0 &&
                    threadIdx.y - FILTER_RADIUS + f_row < TILE_DIM)
                {
                    p_value += s_N[threadIdx.y - FILTER_RADIUS + f_row]
                                  [threadIdx.x - FILTER_RADIUS + f_col] *
                               dc_F[f_row * (2 * FILTER_RADIUS + 1) + f_col];
                }
                else
                { // load from global memory
                    if (row - FILTER_RADIUS + f_row >= 0 &&
                        row - FILTER_RADIUS + f_row < height &&
                        col - FILTER_RADIUS + f_col >= 0 &&
                        col - FILTER_RADIUS + f_col < width)
                    {
                        p_value += N[(row - FILTER_RADIUS + f_row) * width + (col - FILTER_RADIUS + f_col)] *
                                   dc_F[f_row * (2 * FILTER_RADIUS + 1) + f_col];
                    }
                }
            }
        }
        P[row * width + col] = p_value;
    }
}

void call_basic_kernel(float *N, float *F, float *P, int width, int height)
{
    // Device memory pointers
    float *d_N, *d_F, *d_P;

    // Allocate device memory for input tensor
    cudaMalloc(&d_N, width * height * sizeof(float));
    // Allocate device memory for filter tensor
    cudaMalloc(&d_F, (2 * FILTER_RADIUS + 1) * (2 * FILTER_RADIUS + 1) * sizeof(float));
    // Allocate device memory for output tensor
    cudaMalloc(&d_P, width * height * sizeof(float));

    // Copy input tensor to device memory
    cudaMemcpy(d_N, N, width * height * sizeof(float), cudaMemcpyHostToDevice);
    // Copy filter tensor to device memory
    cudaMemcpy(d_F, F, (2 * FILTER_RADIUS + 1) * (2 * FILTER_RADIUS + 1) * sizeof(float), cudaMemcpyHostToDevice);

    // Configure block and grid sizes
    dim3 block_size(32, 32);
    dim3 grid_size((width + block_size.x - 1) / block_size.x, (height + block_size.y - 1) / block_size.y);

    // Create CUDA events for timing
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // Warm-up kernel execution (optional, avoids first-time overhead)
    convolution_2d_basic_kernel<<<grid_size, block_size>>>(d_N, d_F, d_P, FILTER_RADIUS, width, height);
    cudaDeviceSynchronize();

    // Start recording time
    cudaEventRecord(start, 0);

    // Execute the kernel 10 times for benchmarking
    for (int i = 0; i < 10; i++)
    {
        convolution_2d_basic_kernel<<<grid_size, block_size>>>(d_N, d_F, d_P, FILTER_RADIUS, width, height);
    }
    cudaDeviceSynchronize();

    // Stop recording time
    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);

    // Calculate elapsed time
    float elapsed_time;
    cudaEventElapsedTime(&elapsed_time, start, stop);

    // Print the average time per kernel execution
    printf("Average time per convolution_2d_basic_kernel: %.3f ms\n", elapsed_time / 10);

    // Copy output tensor back to host memory
    cudaMemcpy(P, d_P, width * height * sizeof(float), cudaMemcpyDeviceToHost);

    // verification
    verify_convolution_result(N, F, P, width, height, FILTER_RADIUS);

    // Print the first 10 elements of the output tensor
    for (int i = 0; i < 10; i++)
    {
        printf("%f, ", P[i]);
    }
    printf("\n");

    // Free device memory
    cudaFree(d_N);
    cudaFree(d_F);
    cudaFree(d_P);

    // Destroy CUDA events
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
}

void call_constant_mem_kernel(float *N, float *F, float *P, int width, int height)
{
    // Device memory pointers
    float *d_N, *d_P;

    // Allocate device memory
    cudaMalloc(&d_N, width * height * sizeof(float));
    cudaMalloc(&d_P, width * height * sizeof(float));

    // Copy data to device memory
    cudaMemcpy(d_N, N, width * height * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpyToSymbol(dc_F, F, (2 * FILTER_RADIUS + 1) * (2 * FILTER_RADIUS + 1) * sizeof(float));

    // Configure kernel launch parameters
    dim3 block_size(32, 32);
    dim3 grid_size((width + block_size.x - 1) / block_size.x,
                   (height + block_size.y - 1) / block_size.y);

    // Create CUDA events
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // Warm-up execution (optional, avoids first-time overhead)
    convolution_2d_constant_mem_kernel<<<grid_size, block_size>>>(d_N, d_P, FILTER_RADIUS, width, height);
    cudaDeviceSynchronize();

    // Start recording time
    cudaEventRecord(start, 0);

    // Execute the kernel 10 times
    for (int i = 0; i < 10; i++)
    {
        convolution_2d_constant_mem_kernel<<<grid_size, block_size>>>(d_N, d_P, FILTER_RADIUS, width, height);
    }
    cudaDeviceSynchronize();

    // Stop recording time
    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);

    // Calculate elapsed time
    float elapsed_time;
    cudaEventElapsedTime(&elapsed_time, start, stop);

    // Print average time per kernel execution
    printf("Average time per convolution_2d_constant_mem_kernel: %.3f ms\n", elapsed_time / 10);

    // Copy output data back to host memory
    cudaMemcpy(P, d_P, width * height * sizeof(float), cudaMemcpyDeviceToHost);

    // verification
    verify_convolution_result(N, F, P, width, height, FILTER_RADIUS);

    // Print first 10 elements of output tensor
    for (int i = 0; i < 10; i++)
    {
        printf("%f, ", P[i]);
    }
    printf("\n");

    // Free device memory and destroy CUDA events
    cudaFree(d_N);
    cudaFree(d_P);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
}

void call_2d_tiled_constant_mem_kernel(float *N, float *F, float *P, int width, int height)
{
    // Device memory pointers
    float *d_N, *d_P;

    // Allocate device memory for input tensor
    cudaMalloc(&d_N, width * height * sizeof(float));
    // Allocate device memory for output tensor
    cudaMalloc(&d_P, width * height * sizeof(float));

    // Copy input tensor to device memory
    cudaMemcpy(d_N, N, width * height * sizeof(float), cudaMemcpyHostToDevice);
    // Copy filter tensor to constant memory
    cudaMemcpyToSymbol(dc_F, F, (2 * FILTER_RADIUS + 1) * (2 * FILTER_RADIUS + 1) * sizeof(float));

    // Block and grid size configuration
    dim3 block_size(IN_TILE_DIM, IN_TILE_DIM);
    dim3 grid_size((width + OUT_TILE_DIM - 1) / OUT_TILE_DIM,
                   (height + OUT_TILE_DIM - 1) / OUT_TILE_DIM);

    // Create CUDA events for timing
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // Warm-up execution to avoid first-time overhead
    convolution_2d_tiled_const_mem_kernel<<<grid_size, block_size>>>(d_N, d_P, width, height);
    cudaDeviceSynchronize();

    // Start recording time
    cudaEventRecord(start, 0);

    // Execute the kernel 10 times for averaging
    for (int i = 0; i < 10; i++)
    {
        convolution_2d_tiled_const_mem_kernel<<<grid_size, block_size>>>(d_N, d_P, width, height);
    }
    cudaDeviceSynchronize();

    // Stop recording time
    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);

    // Calculate elapsed time
    float elapsed_time;
    cudaEventElapsedTime(&elapsed_time, start, stop);

    // Print the average time per kernel execution
    printf("Average time per convolution_2d_tiled_const_mem_kernel: %.3f ms\n", elapsed_time / 10);

    // Copy output tensor to host memory
    cudaMemcpy(P, d_P, width * height * sizeof(float), cudaMemcpyDeviceToHost);

    // verification
    verify_convolution_result(N, F, P, width, height, FILTER_RADIUS);

    // Print the first 10 elements of the output tensor
    for (int i = 0; i < 10; i++)
    {
        printf("%f, ", P[i]);
    }
    printf("\n");

    // Free device memory
    cudaFree(d_N);
    cudaFree(d_P);

    // Destroy CUDA events
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
}

void call_2d_cached_tiled_constant_mem_kernel(float *N, float *F, float *P, int width, int height)
{
    // Device memory pointers
    float *d_N, *d_P;

    // Allocate device memory for input tensor
    cudaMalloc(&d_N, width * height * sizeof(float));
    // Allocate device memory for output tensor
    cudaMalloc(&d_P, width * height * sizeof(float));

    // Copy input tensor to device memory
    cudaMemcpy(d_N, N, width * height * sizeof(float), cudaMemcpyHostToDevice);
    // Copy filter tensor to constant memory
    cudaMemcpyToSymbol(dc_F, F, (2 * FILTER_RADIUS + 1) * (2 * FILTER_RADIUS + 1) * sizeof(float));

    // Configure block and grid size
    dim3 block_size(TILE_DIM, TILE_DIM);
    dim3 grid_size((width + TILE_DIM - 1) / TILE_DIM, (height + TILE_DIM - 1) / TILE_DIM);

    // Create CUDA events for timing
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // Warm-up execution (optional, avoids first-time overhead)
    convolution_2d_cached_tiled_const_mem_kernel<<<grid_size, block_size>>>(d_N, d_P, width, height);
    cudaDeviceSynchronize();

    // Start recording time
    cudaEventRecord(start, 0);

    // Execute the kernel 10 times for averaging
    for (int i = 0; i < 10; i++)
    {
        convolution_2d_cached_tiled_const_mem_kernel<<<grid_size, block_size>>>(d_N, d_P, width, height);
    }
    cudaDeviceSynchronize();

    // Stop recording time
    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);

    // Calculate elapsed time
    float elapsed_time;
    cudaEventElapsedTime(&elapsed_time, start, stop);

    // Print the average time per kernel execution
    printf("Average time per convolution_2d_cached_tiled_const_mem_kernel: %.3f ms\n", elapsed_time / 10);

    // Copy output tensor to host memory
    cudaMemcpy(P, d_P, width * height * sizeof(float), cudaMemcpyDeviceToHost);

    // verification
    verify_convolution_result(N, F, P, width, height, FILTER_RADIUS);

    // Print the first 10 elements of the output tensor
    for (int i = 0; i < 10; i++)
    {
        printf("%f, ", P[i]);
    }
    printf("\n");

    // Free device memory
    cudaFree(d_N);
    cudaFree(d_P);

    // Destroy CUDA events
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
}

int main(int argc, char **argv)
{

    // input tensor
    float *N;
    // filter tensor
    float *F;
    // output tensor
    float *P;

    // size of input tensor
    int width = 8192;
    int height = 8192;

    // allocate memory for input tensor
    N = (float *)malloc(width * height * sizeof(float));
    // allocate memory for filter tensor
    F = (float *)malloc((2 * FILTER_RADIUS + 1) * (2 * FILTER_RADIUS + 1) * sizeof(float));
    // allocate memory for output tensor
    P = (float *)malloc(width * height * sizeof(float));

    // initialize input tensor
    for (int i = 0; i < width * height; i++)
    {
        N[i] = i * 0.1f;
    }

    // initialize filter tensor
    for (int i = 0; i < (2 * FILTER_RADIUS + 1) * (2 * FILTER_RADIUS + 1); i++)
    {
        F[i] = i * 0.05f;
    }

    // call the basic kernel
    call_basic_kernel(N, F, P, width, height);

    // call the kernel with constant memory
    call_constant_mem_kernel(N, F, P, width, height);

    // call the tiled kernel
    call_2d_tiled_constant_mem_kernel(N, F, P, width, height);

    // call the cached tiled kernel
    call_2d_cached_tiled_constant_mem_kernel(N, F, P, width, height);

    // free memory
    free(N);
    free(F);
    free(P);

    return 0;
}

Average time per convolution_2d_basic_kernel: 12.710 ms
Verification Results:
Max error: 2.400000e+01
Average error: 1.313119e+00
Result differs more than the threshold!
7865.159668, 10241.650391, 12495.500977, 12496.775391, 12498.050781, 12499.325195, 12500.600586, 12501.875977, 12503.151367, 12504.425781, 
Average time per convolution_2d_constant_mem_kernel: 11.244 ms
Verification Results:
Max error: 2.400000e+01
Average error: 1.313119e+00
Result differs more than the threshold!
7865.159668, 10241.650391, 12495.500977, 12496.775391, 12498.050781, 12499.325195, 12500.600586, 12501.875977, 12503.151367, 12504.425781, 
Average time per convolution_2d_tiled_const_mem_kernel: 9.489 ms
Verification Results:
Max error: 2.400000e+01
Average error: 1.313119e+00
Result differs more than the threshold!
7865.159668, 10241.650391, 12495.500977, 12496.775391, 12498.050781, 12499.325195, 12500.600586, 12501.875977, 12503.151367, 12504.425781, 
Average time per convolution_2d_cached_tiled_const_mem

### result

## Tile = 16

In [ ]:
%%cuda

#include <cuda.h>
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <math.h>

#define MASK_SIZE 5
#define FILTER_RADIUS (MASK_SIZE / 2)

// tile size
#define IN_TILE_DIM 16
#define OUT_TILE_DIM (IN_TILE_DIM - 2 * FILTER_RADIUS)

__constant__ float dc_F[(2 * FILTER_RADIUS + 1) * (2 * FILTER_RADIUS + 1)];

// Performing 2D convolution on the CPU to verify the correctness of the GPU results.
void host_convolution_2d(float *N, float *F, float *P, int width, int height, int r)
{
    int filter_dim = 2 * r + 1;
    for (int row = 0; row < height; row++)
    {
        for (int col = 0; col < width; col++)
        {
            float p_value = 0.0f;
            for (int f_row = 0; f_row < filter_dim; f_row++)
            {
                for (int f_col = 0; f_col < filter_dim; f_col++)
                {
                    int n_row = row - r + f_row;
                    int n_col = col - r + f_col;
                    if (n_row >= 0 && n_row < height && n_col >= 0 && n_col < width)
                    {
                        p_value += N[n_row * width + n_col] * F[f_row * filter_dim + f_col];
                    }
                }
            }
            P[row * width + col] = p_value;
        }
    }
}

// Verify the difference between the GPU convolution result and the CPU result.
void verify_convolution_result(float *N, float *F, float *P_gpu, int width, int height, int r, float threshold = 1e0f)
{
    // Allocate CPU output array
    float *P_cpu = (float *)malloc(width * height * sizeof(float));

    // Using CPU to perform convolution
    host_convolution_2d(N, F, P_cpu, width, height, r);

    // Calculation error
    double max_error = 0.0;
    double sum_error = 0.0;
    for (int i = 0; i < width * height; i++)
    {
        double err = fabs(P_cpu[i] - P_gpu[i]);
        if (err > max_error)
        {
            max_error = err;
        }
        sum_error += err;
    }

    double avg_error = sum_error / (width * height);

    printf("Verification Results:\n");
    printf("Max error: %e\n", max_error);
    printf("Average error: %e\n", avg_error);

    if (max_error < threshold)
    {
        printf("Result is correct within the threshold.\n");
    }
    else
    {
        printf("Result differs more than the threshold!\n");
    }

    // Release memory
    free(P_cpu);
}

// basic 2d convolution, each thread compute the each element of the output tensor
__global__ void convolution_2d_basic_kernel(float *N, float *F, float *P, int r, int width, int height)
{
    /*
    N: input tensor
    F: filter tensor
    P: output tensor
    r: filter radius (2r+1)x(2r+1)
    width: width of input tensor
    height: height of input tensor
    N and P has the same size
    */

    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int row = blockIdx.y * blockDim.y + threadIdx.y;

    if (col < width && row < height) {
        float p_value = 0.0;
        for (int f_row = 0; f_row < 2 * r + 1; f_row++)
        {
            for (int f_col = 0; f_col < 2 * r + 1; f_col++)
            {
                int n_row = row - r + f_row;
                int n_col = col - r + f_col;
                if (n_row >= 0 && n_row < height && n_col >= 0 && n_col < width)
                {
                    p_value += N[n_row * width + n_col] * F[f_row * (2 * r + 1) + f_col];
                }
            }
        }
        P[row * width + col] = p_value;
    }
}

// constatn memory version of 2d convolution, each thread compute the each element of the output tensor
__global__ void convolution_2d_constant_mem_kernel(float *N, float *P, int r, int width, int height)
{
    /*
    N: input tensor
    P: output tensor
    r: filter radius (2r+1)x(2r+1)
    width: width of input tensor
    height: height of input tensor
    N and P has the same size
    use constant memory for the filter tensor F
    */

    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int row = blockIdx.y * blockDim.y + threadIdx.y;

    if (col < width && row < height) {
        float p_value = 0.0;
        for (int f_row = 0; f_row < 2 * r + 1; f_row++)
        {
            for (int f_col = 0; f_col < 2 * r + 1; f_col++)
            {
                int n_row = row - r + f_row;
                int n_col = col - r + f_col;
                if (n_row >= 0 && n_row < height && n_col >= 0 && n_col < width)
                {
                    p_value += N[n_row * width + n_col] * dc_F[f_row * (2 * r + 1) + f_col];
                }
            }
        }
        P[row * width + col] = p_value;
    }
}

// tiled version of 2d convolution, a block of threads compute a tile of the output tensor, use the shared memory to cache the input tensor
__global__ void convolution_2d_tiled_const_mem_kernel(float *N, float *P, int width, int height)
{
    /*
    N: input tensor
    P: output tensor
    width: width of input tensor
    height: height of input tensor
    N and P has the same size
    use constant memory for the filter tensor F
    */

    __shared__ float s_N[IN_TILE_DIM][IN_TILE_DIM];

    int col = blockIdx.x * OUT_TILE_DIM + threadIdx.x - FILTER_RADIUS;
    int row = blockIdx.y * OUT_TILE_DIM + threadIdx.y - FILTER_RADIUS;

    if (col >= 0 && col < width && row >= 0 && row < height)
    {
        s_N[threadIdx.y][threadIdx.x] = N[row * width + col];
    }
    else
    {
        s_N[threadIdx.y][threadIdx.x] = 0.0;
    }

    __syncthreads();

    int tile_col = threadIdx.x - FILTER_RADIUS;
    int tile_row = threadIdx.y - FILTER_RADIUS;

    if (col >= 0 && col < width && row >= 0 && row < height)
    {
        if (tile_col >= 0 && tile_col < OUT_TILE_DIM && tile_row >= 0 && tile_row < OUT_TILE_DIM)
        {
            float p_value = 0.0;
            for (int f_row = 0; f_row < 2 * FILTER_RADIUS + 1; f_row++)
            {
                for (int f_col = 0; f_col < 2 * FILTER_RADIUS + 1; f_col++)
                {
                    p_value += s_N[tile_row + f_row][tile_col + f_col] * dc_F[f_row * (2 * FILTER_RADIUS + 1) + f_col];
                }
            }
            P[row * width + col] = p_value;
        }
    }
}

#define TILE_DIM 16
// tiled version of 2d convolution, a block of threads compute a tile of the output tensor, use the shared memory to cache the input tensor
__global__ void convolution_2d_cached_tiled_const_mem_kernel(float *N, float *P,
                                                             int width,
                                                             int height)
{
    /*
    N: input tensor
    P: output tensor
    width: width of input tensor
    height: height of input tensor
    N and P has the same size
    use constant memory for the filter tensor F
    */

    __shared__ float s_N[TILE_DIM][TILE_DIM];

    int col = blockIdx.x * TILE_DIM + threadIdx.x;
    int row = blockIdx.y * TILE_DIM + threadIdx.y;
    if (col >= 0 && col < width && row >= 0 && row < height)
    {
        s_N[threadIdx.y][threadIdx.x] = N[row * width + col];
    }
    else
    {
        s_N[threadIdx.y][threadIdx.x] = 0.0;
    }

    __syncthreads();

    if (col < width && row < height)
    {
        float p_value = 0.0;

        for (int f_row = 0; f_row < 2 * FILTER_RADIUS + 1; f_row++)
        {
            for (int f_col = 0; f_col < 2 * FILTER_RADIUS + 1; f_col++)
            {
                if (threadIdx.x - FILTER_RADIUS + f_col >= 0 &&
                    threadIdx.x - FILTER_RADIUS + f_col < TILE_DIM &&
                    threadIdx.y - FILTER_RADIUS + f_row >= 0 &&
                    threadIdx.y - FILTER_RADIUS + f_row < TILE_DIM)
                {
                    p_value += s_N[threadIdx.y - FILTER_RADIUS + f_row]
                                  [threadIdx.x - FILTER_RADIUS + f_col] *
                               dc_F[f_row * (2 * FILTER_RADIUS + 1) + f_col];
                }
                else
                { // load from global memory
                    if (row - FILTER_RADIUS + f_row >= 0 &&
                        row - FILTER_RADIUS + f_row < height &&
                        col - FILTER_RADIUS + f_col >= 0 &&
                        col - FILTER_RADIUS + f_col < width)
                    {
                        p_value += N[(row - FILTER_RADIUS + f_row) * width + (col - FILTER_RADIUS + f_col)] *
                                   dc_F[f_row * (2 * FILTER_RADIUS + 1) + f_col];
                    }
                }
            }
        }
        P[row * width + col] = p_value;
    }
}

void call_basic_kernel(float *N, float *F, float *P, int width, int height)
{
    // Device memory pointers
    float *d_N, *d_F, *d_P;

    // Allocate device memory for input tensor
    cudaMalloc(&d_N, width * height * sizeof(float));
    // Allocate device memory for filter tensor
    cudaMalloc(&d_F, (2 * FILTER_RADIUS + 1) * (2 * FILTER_RADIUS + 1) * sizeof(float));
    // Allocate device memory for output tensor
    cudaMalloc(&d_P, width * height * sizeof(float));

    // Copy input tensor to device memory
    cudaMemcpy(d_N, N, width * height * sizeof(float), cudaMemcpyHostToDevice);
    // Copy filter tensor to device memory
    cudaMemcpy(d_F, F, (2 * FILTER_RADIUS + 1) * (2 * FILTER_RADIUS + 1) * sizeof(float), cudaMemcpyHostToDevice);

    // Configure block and grid sizes
    dim3 block_size(32, 32);
    dim3 grid_size((width + block_size.x - 1) / block_size.x, (height + block_size.y - 1) / block_size.y);

    // Create CUDA events for timing
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // Warm-up kernel execution (optional, avoids first-time overhead)
    convolution_2d_basic_kernel<<<grid_size, block_size>>>(d_N, d_F, d_P, FILTER_RADIUS, width, height);
    cudaDeviceSynchronize();

    // Start recording time
    cudaEventRecord(start, 0);

    // Execute the kernel 10 times for benchmarking
    for (int i = 0; i < 10; i++)
    {
        convolution_2d_basic_kernel<<<grid_size, block_size>>>(d_N, d_F, d_P, FILTER_RADIUS, width, height);
    }
    cudaDeviceSynchronize();

    // Stop recording time
    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);

    // Calculate elapsed time
    float elapsed_time;
    cudaEventElapsedTime(&elapsed_time, start, stop);

    // Print the average time per kernel execution
    printf("Average time per convolution_2d_basic_kernel: %.3f ms\n", elapsed_time / 10);

    // Copy output tensor back to host memory
    cudaMemcpy(P, d_P, width * height * sizeof(float), cudaMemcpyDeviceToHost);

    // verification
    verify_convolution_result(N, F, P, width, height, FILTER_RADIUS);

    // Print the first 10 elements of the output tensor
    for (int i = 0; i < 10; i++)
    {
        printf("%f, ", P[i]);
    }
    printf("\n");

    // Free device memory
    cudaFree(d_N);
    cudaFree(d_F);
    cudaFree(d_P);

    // Destroy CUDA events
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
}

void call_constant_mem_kernel(float *N, float *F, float *P, int width, int height)
{
    // Device memory pointers
    float *d_N, *d_P;

    // Allocate device memory
    cudaMalloc(&d_N, width * height * sizeof(float));
    cudaMalloc(&d_P, width * height * sizeof(float));

    // Copy data to device memory
    cudaMemcpy(d_N, N, width * height * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpyToSymbol(dc_F, F, (2 * FILTER_RADIUS + 1) * (2 * FILTER_RADIUS + 1) * sizeof(float));

    // Configure kernel launch parameters
    dim3 block_size(32, 32);
    dim3 grid_size((width + block_size.x - 1) / block_size.x,
                   (height + block_size.y - 1) / block_size.y);

    // Create CUDA events
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // Warm-up execution (optional, avoids first-time overhead)
    convolution_2d_constant_mem_kernel<<<grid_size, block_size>>>(d_N, d_P, FILTER_RADIUS, width, height);
    cudaDeviceSynchronize();

    // Start recording time
    cudaEventRecord(start, 0);

    // Execute the kernel 10 times
    for (int i = 0; i < 10; i++)
    {
        convolution_2d_constant_mem_kernel<<<grid_size, block_size>>>(d_N, d_P, FILTER_RADIUS, width, height);
    }
    cudaDeviceSynchronize();

    // Stop recording time
    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);

    // Calculate elapsed time
    float elapsed_time;
    cudaEventElapsedTime(&elapsed_time, start, stop);

    // Print average time per kernel execution
    printf("Average time per convolution_2d_constant_mem_kernel: %.3f ms\n", elapsed_time / 10);

    // Copy output data back to host memory
    cudaMemcpy(P, d_P, width * height * sizeof(float), cudaMemcpyDeviceToHost);

    // verification
    verify_convolution_result(N, F, P, width, height, FILTER_RADIUS);

    // Print first 10 elements of output tensor
    for (int i = 0; i < 10; i++)
    {
        printf("%f, ", P[i]);
    }
    printf("\n");

    // Free device memory and destroy CUDA events
    cudaFree(d_N);
    cudaFree(d_P);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
}

void call_2d_tiled_constant_mem_kernel(float *N, float *F, float *P, int width, int height)
{
    // Device memory pointers
    float *d_N, *d_P;

    // Allocate device memory for input tensor
    cudaMalloc(&d_N, width * height * sizeof(float));
    // Allocate device memory for output tensor
    cudaMalloc(&d_P, width * height * sizeof(float));

    // Copy input tensor to device memory
    cudaMemcpy(d_N, N, width * height * sizeof(float), cudaMemcpyHostToDevice);
    // Copy filter tensor to constant memory
    cudaMemcpyToSymbol(dc_F, F, (2 * FILTER_RADIUS + 1) * (2 * FILTER_RADIUS + 1) * sizeof(float));

    // Block and grid size configuration
    dim3 block_size(IN_TILE_DIM, IN_TILE_DIM);
    dim3 grid_size((width + OUT_TILE_DIM - 1) / OUT_TILE_DIM,
                   (height + OUT_TILE_DIM - 1) / OUT_TILE_DIM);

    // Create CUDA events for timing
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // Warm-up execution to avoid first-time overhead
    convolution_2d_tiled_const_mem_kernel<<<grid_size, block_size>>>(d_N, d_P, width, height);
    cudaDeviceSynchronize();

    // Start recording time
    cudaEventRecord(start, 0);

    // Execute the kernel 10 times for averaging
    for (int i = 0; i < 10; i++)
    {
        convolution_2d_tiled_const_mem_kernel<<<grid_size, block_size>>>(d_N, d_P, width, height);
    }
    cudaDeviceSynchronize();

    // Stop recording time
    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);

    // Calculate elapsed time
    float elapsed_time;
    cudaEventElapsedTime(&elapsed_time, start, stop);

    // Print the average time per kernel execution
    printf("Average time per convolution_2d_tiled_const_mem_kernel: %.3f ms\n", elapsed_time / 10);

    // Copy output tensor to host memory
    cudaMemcpy(P, d_P, width * height * sizeof(float), cudaMemcpyDeviceToHost);

    // verification
    verify_convolution_result(N, F, P, width, height, FILTER_RADIUS);

    // Print the first 10 elements of the output tensor
    for (int i = 0; i < 10; i++)
    {
        printf("%f, ", P[i]);
    }
    printf("\n");

    // Free device memory
    cudaFree(d_N);
    cudaFree(d_P);

    // Destroy CUDA events
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
}

void call_2d_cached_tiled_constant_mem_kernel(float *N, float *F, float *P, int width, int height)
{
    // Device memory pointers
    float *d_N, *d_P;

    // Allocate device memory for input tensor
    cudaMalloc(&d_N, width * height * sizeof(float));
    // Allocate device memory for output tensor
    cudaMalloc(&d_P, width * height * sizeof(float));

    // Copy input tensor to device memory
    cudaMemcpy(d_N, N, width * height * sizeof(float), cudaMemcpyHostToDevice);
    // Copy filter tensor to constant memory
    cudaMemcpyToSymbol(dc_F, F, (2 * FILTER_RADIUS + 1) * (2 * FILTER_RADIUS + 1) * sizeof(float));

    // Configure block and grid size
    dim3 block_size(TILE_DIM, TILE_DIM);
    dim3 grid_size((width + TILE_DIM - 1) / TILE_DIM, (height + TILE_DIM - 1) / TILE_DIM);

    // Create CUDA events for timing
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // Warm-up execution (optional, avoids first-time overhead)
    convolution_2d_cached_tiled_const_mem_kernel<<<grid_size, block_size>>>(d_N, d_P, width, height);
    cudaDeviceSynchronize();

    // Start recording time
    cudaEventRecord(start, 0);

    // Execute the kernel 10 times for averaging
    for (int i = 0; i < 10; i++)
    {
        convolution_2d_cached_tiled_const_mem_kernel<<<grid_size, block_size>>>(d_N, d_P, width, height);
    }
    cudaDeviceSynchronize();

    // Stop recording time
    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);

    // Calculate elapsed time
    float elapsed_time;
    cudaEventElapsedTime(&elapsed_time, start, stop);

    // Print the average time per kernel execution
    printf("Average time per convolution_2d_cached_tiled_const_mem_kernel: %.3f ms\n", elapsed_time / 10);

    // Copy output tensor to host memory
    cudaMemcpy(P, d_P, width * height * sizeof(float), cudaMemcpyDeviceToHost);

    // verification
    verify_convolution_result(N, F, P, width, height, FILTER_RADIUS);

    // Print the first 10 elements of the output tensor
    for (int i = 0; i < 10; i++)
    {
        printf("%f, ", P[i]);
    }
    printf("\n");

    // Free device memory
    cudaFree(d_N);
    cudaFree(d_P);

    // Destroy CUDA events
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
}

int main(int argc, char **argv)
{

    // input tensor
    float *N;
    // filter tensor
    float *F;
    // output tensor
    float *P;

    // size of input tensor
    int width = 8192;
    int height = 8192;

    // allocate memory for input tensor
    N = (float *)malloc(width * height * sizeof(float));
    // allocate memory for filter tensor
    F = (float *)malloc((2 * FILTER_RADIUS + 1) * (2 * FILTER_RADIUS + 1) * sizeof(float));
    // allocate memory for output tensor
    P = (float *)malloc(width * height * sizeof(float));

    // initialize input tensor
    for (int i = 0; i < width * height; i++)
    {
        N[i] = i * 0.1f;
    }

    // initialize filter tensor
    for (int i = 0; i < (2 * FILTER_RADIUS + 1) * (2 * FILTER_RADIUS + 1); i++)
    {
        F[i] = i * 0.05f;
    }

    // call the basic kernel
    call_basic_kernel(N, F, P, width, height);

    // call the kernel with constant memory
    call_constant_mem_kernel(N, F, P, width, height);

    // call the tiled kernel
    call_2d_tiled_constant_mem_kernel(N, F, P, width, height);

    // call the cached tiled kernel
    call_2d_cached_tiled_constant_mem_kernel(N, F, P, width, height);

    // free memory
    free(N);
    free(F);
    free(P);

    return 0;
}

Average time per convolution_2d_basic_kernel: 13.989 ms
Verification Results:
Max error: 2.400000e+01
Average error: 1.313119e+00
Result differs more than the threshold!
7865.159668, 10241.650391, 12495.500977, 12496.775391, 12498.050781, 12499.325195, 12500.600586, 12501.875977, 12503.151367, 12504.425781, 
Average time per convolution_2d_constant_mem_kernel: 12.427 ms
Verification Results:
Max error: 2.400000e+01
Average error: 1.313119e+00
Result differs more than the threshold!
7865.159668, 10241.650391, 12495.500977, 12496.775391, 12498.050781, 12499.325195, 12500.600586, 12501.875977, 12503.151367, 12504.425781, 
Average time per convolution_2d_tiled_const_mem_kernel: 8.521 ms
Verification Results:
Max error: 2.400000e+01
Average error: 1.313119e+00
Result differs more than the threshold!
7865.159668, 10241.650391, 12495.500977, 12496.775391, 12498.050781, 12499.325195, 12500.600586, 12501.875977, 12503.151367, 12504.425781, 
Average time per convolution_2d_cached_tiled_const_mem

### result

## Tile = 8

In [ ]:
%%cuda

#include <cuda.h>
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <math.h>

#define MASK_SIZE 5
#define FILTER_RADIUS (MASK_SIZE / 2)

// tile size
#define IN_TILE_DIM 8
#define OUT_TILE_DIM (IN_TILE_DIM - 2 * FILTER_RADIUS)

__constant__ float dc_F[(2 * FILTER_RADIUS + 1) * (2 * FILTER_RADIUS + 1)];

// Performing 2D convolution on the CPU to verify the correctness of the GPU results.
void host_convolution_2d(float *N, float *F, float *P, int width, int height, int r)
{
    int filter_dim = 2 * r + 1;
    for (int row = 0; row < height; row++)
    {
        for (int col = 0; col < width; col++)
        {
            float p_value = 0.0f;
            for (int f_row = 0; f_row < filter_dim; f_row++)
            {
                for (int f_col = 0; f_col < filter_dim; f_col++)
                {
                    int n_row = row - r + f_row;
                    int n_col = col - r + f_col;
                    if (n_row >= 0 && n_row < height && n_col >= 0 && n_col < width)
                    {
                        p_value += N[n_row * width + n_col] * F[f_row * filter_dim + f_col];
                    }
                }
            }
            P[row * width + col] = p_value;
        }
    }
}

// Verify the difference between the GPU convolution result and the CPU result.
void verify_convolution_result(float *N, float *F, float *P_gpu, int width, int height, int r, float threshold = 1e0f)
{
    // Allocate CPU output array
    float *P_cpu = (float *)malloc(width * height * sizeof(float));

    // Using CPU to perform convolution
    host_convolution_2d(N, F, P_cpu, width, height, r);

    // Calculation error
    double max_error = 0.0;
    double sum_error = 0.0;
    for (int i = 0; i < width * height; i++)
    {
        double err = fabs(P_cpu[i] - P_gpu[i]);
        if (err > max_error)
        {
            max_error = err;
        }
        sum_error += err;
    }

    double avg_error = sum_error / (width * height);

    printf("Verification Results:\n");
    printf("Max error: %e\n", max_error);
    printf("Average error: %e\n", avg_error);

    if (max_error < threshold)
    {
        printf("Result is correct within the threshold.\n");
    }
    else
    {
        printf("Result differs more than the threshold!\n");
    }

    // Release memory
    free(P_cpu);
}

// basic 2d convolution, each thread compute the each element of the output tensor
__global__ void convolution_2d_basic_kernel(float *N, float *F, float *P, int r, int width, int height)
{
    /*
    N: input tensor
    F: filter tensor
    P: output tensor
    r: filter radius (2r+1)x(2r+1)
    width: width of input tensor
    height: height of input tensor
    N and P has the same size
    */

    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int row = blockIdx.y * blockDim.y + threadIdx.y;

    if (col < width && row < height) {
        float p_value = 0.0;
        for (int f_row = 0; f_row < 2 * r + 1; f_row++)
        {
            for (int f_col = 0; f_col < 2 * r + 1; f_col++)
            {
                int n_row = row - r + f_row;
                int n_col = col - r + f_col;
                if (n_row >= 0 && n_row < height && n_col >= 0 && n_col < width)
                {
                    p_value += N[n_row * width + n_col] * F[f_row * (2 * r + 1) + f_col];
                }
            }
        }
        P[row * width + col] = p_value;
    }
}

// constatn memory version of 2d convolution, each thread compute the each element of the output tensor
__global__ void convolution_2d_constant_mem_kernel(float *N, float *P, int r, int width, int height)
{
    /*
    N: input tensor
    P: output tensor
    r: filter radius (2r+1)x(2r+1)
    width: width of input tensor
    height: height of input tensor
    N and P has the same size
    use constant memory for the filter tensor F
    */

    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int row = blockIdx.y * blockDim.y + threadIdx.y;

    if (col < width && row < height) {
        float p_value = 0.0;
        for (int f_row = 0; f_row < 2 * r + 1; f_row++)
        {
            for (int f_col = 0; f_col < 2 * r + 1; f_col++)
            {
                int n_row = row - r + f_row;
                int n_col = col - r + f_col;
                if (n_row >= 0 && n_row < height && n_col >= 0 && n_col < width)
                {
                    p_value += N[n_row * width + n_col] * dc_F[f_row * (2 * r + 1) + f_col];
                }
            }
        }
        P[row * width + col] = p_value;
    }
}

// tiled version of 2d convolution, a block of threads compute a tile of the output tensor, use the shared memory to cache the input tensor
__global__ void convolution_2d_tiled_const_mem_kernel(float *N, float *P, int width, int height)
{
    /*
    N: input tensor
    P: output tensor
    width: width of input tensor
    height: height of input tensor
    N and P has the same size
    use constant memory for the filter tensor F
    */

    __shared__ float s_N[IN_TILE_DIM][IN_TILE_DIM];

    int col = blockIdx.x * OUT_TILE_DIM + threadIdx.x - FILTER_RADIUS;
    int row = blockIdx.y * OUT_TILE_DIM + threadIdx.y - FILTER_RADIUS;

    if (col >= 0 && col < width && row >= 0 && row < height)
    {
        s_N[threadIdx.y][threadIdx.x] = N[row * width + col];
    }
    else
    {
        s_N[threadIdx.y][threadIdx.x] = 0.0;
    }

    __syncthreads();

    int tile_col = threadIdx.x - FILTER_RADIUS;
    int tile_row = threadIdx.y - FILTER_RADIUS;

    if (col >= 0 && col < width && row >= 0 && row < height)
    {
        if (tile_col >= 0 && tile_col < OUT_TILE_DIM && tile_row >= 0 && tile_row < OUT_TILE_DIM)
        {
            float p_value = 0.0;
            for (int f_row = 0; f_row < 2 * FILTER_RADIUS + 1; f_row++)
            {
                for (int f_col = 0; f_col < 2 * FILTER_RADIUS + 1; f_col++)
                {
                    p_value += s_N[tile_row + f_row][tile_col + f_col] * dc_F[f_row * (2 * FILTER_RADIUS + 1) + f_col];
                }
            }
            P[row * width + col] = p_value;
        }
    }
}

#define TILE_DIM 8
// tiled version of 2d convolution, a block of threads compute a tile of the output tensor, use the shared memory to cache the input tensor
__global__ void convolution_2d_cached_tiled_const_mem_kernel(float *N, float *P,
                                                             int width,
                                                             int height)
{
    /*
    N: input tensor
    P: output tensor
    width: width of input tensor
    height: height of input tensor
    N and P has the same size
    use constant memory for the filter tensor F
    */

    __shared__ float s_N[TILE_DIM][TILE_DIM];

    int col = blockIdx.x * TILE_DIM + threadIdx.x;
    int row = blockIdx.y * TILE_DIM + threadIdx.y;
    if (col >= 0 && col < width && row >= 0 && row < height)
    {
        s_N[threadIdx.y][threadIdx.x] = N[row * width + col];
    }
    else
    {
        s_N[threadIdx.y][threadIdx.x] = 0.0;
    }

    __syncthreads();

    if (col < width && row < height)
    {
        float p_value = 0.0;

        for (int f_row = 0; f_row < 2 * FILTER_RADIUS + 1; f_row++)
        {
            for (int f_col = 0; f_col < 2 * FILTER_RADIUS + 1; f_col++)
            {
                if (threadIdx.x - FILTER_RADIUS + f_col >= 0 &&
                    threadIdx.x - FILTER_RADIUS + f_col < TILE_DIM &&
                    threadIdx.y - FILTER_RADIUS + f_row >= 0 &&
                    threadIdx.y - FILTER_RADIUS + f_row < TILE_DIM)
                {
                    p_value += s_N[threadIdx.y - FILTER_RADIUS + f_row]
                                  [threadIdx.x - FILTER_RADIUS + f_col] *
                               dc_F[f_row * (2 * FILTER_RADIUS + 1) + f_col];
                }
                else
                { // load from global memory
                    if (row - FILTER_RADIUS + f_row >= 0 &&
                        row - FILTER_RADIUS + f_row < height &&
                        col - FILTER_RADIUS + f_col >= 0 &&
                        col - FILTER_RADIUS + f_col < width)
                    {
                        p_value += N[(row - FILTER_RADIUS + f_row) * width + (col - FILTER_RADIUS + f_col)] *
                                   dc_F[f_row * (2 * FILTER_RADIUS + 1) + f_col];
                    }
                }
            }
        }
        P[row * width + col] = p_value;
    }
}

void call_basic_kernel(float *N, float *F, float *P, int width, int height)
{
    // Device memory pointers
    float *d_N, *d_F, *d_P;

    // Allocate device memory for input tensor
    cudaMalloc(&d_N, width * height * sizeof(float));
    // Allocate device memory for filter tensor
    cudaMalloc(&d_F, (2 * FILTER_RADIUS + 1) * (2 * FILTER_RADIUS + 1) * sizeof(float));
    // Allocate device memory for output tensor
    cudaMalloc(&d_P, width * height * sizeof(float));

    // Copy input tensor to device memory
    cudaMemcpy(d_N, N, width * height * sizeof(float), cudaMemcpyHostToDevice);
    // Copy filter tensor to device memory
    cudaMemcpy(d_F, F, (2 * FILTER_RADIUS + 1) * (2 * FILTER_RADIUS + 1) * sizeof(float), cudaMemcpyHostToDevice);

    // Configure block and grid sizes
    dim3 block_size(32, 32);
    dim3 grid_size((width + block_size.x - 1) / block_size.x, (height + block_size.y - 1) / block_size.y);

    // Create CUDA events for timing
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // Warm-up kernel execution (optional, avoids first-time overhead)
    convolution_2d_basic_kernel<<<grid_size, block_size>>>(d_N, d_F, d_P, FILTER_RADIUS, width, height);
    cudaDeviceSynchronize();

    // Start recording time
    cudaEventRecord(start, 0);

    // Execute the kernel 10 times for benchmarking
    for (int i = 0; i < 10; i++)
    {
        convolution_2d_basic_kernel<<<grid_size, block_size>>>(d_N, d_F, d_P, FILTER_RADIUS, width, height);
    }
    cudaDeviceSynchronize();

    // Stop recording time
    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);

    // Calculate elapsed time
    float elapsed_time;
    cudaEventElapsedTime(&elapsed_time, start, stop);

    // Print the average time per kernel execution
    printf("Average time per convolution_2d_basic_kernel: %.3f ms\n", elapsed_time / 10);

    // Copy output tensor back to host memory
    cudaMemcpy(P, d_P, width * height * sizeof(float), cudaMemcpyDeviceToHost);

    // verification
    verify_convolution_result(N, F, P, width, height, FILTER_RADIUS);

    // Print the first 10 elements of the output tensor
    for (int i = 0; i < 10; i++)
    {
        printf("%f, ", P[i]);
    }
    printf("\n");

    // Free device memory
    cudaFree(d_N);
    cudaFree(d_F);
    cudaFree(d_P);

    // Destroy CUDA events
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
}

void call_constant_mem_kernel(float *N, float *F, float *P, int width, int height)
{
    // Device memory pointers
    float *d_N, *d_P;

    // Allocate device memory
    cudaMalloc(&d_N, width * height * sizeof(float));
    cudaMalloc(&d_P, width * height * sizeof(float));

    // Copy data to device memory
    cudaMemcpy(d_N, N, width * height * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpyToSymbol(dc_F, F, (2 * FILTER_RADIUS + 1) * (2 * FILTER_RADIUS + 1) * sizeof(float));

    // Configure kernel launch parameters
    dim3 block_size(32, 32);
    dim3 grid_size((width + block_size.x - 1) / block_size.x,
                   (height + block_size.y - 1) / block_size.y);

    // Create CUDA events
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // Warm-up execution (optional, avoids first-time overhead)
    convolution_2d_constant_mem_kernel<<<grid_size, block_size>>>(d_N, d_P, FILTER_RADIUS, width, height);
    cudaDeviceSynchronize();

    // Start recording time
    cudaEventRecord(start, 0);

    // Execute the kernel 10 times
    for (int i = 0; i < 10; i++)
    {
        convolution_2d_constant_mem_kernel<<<grid_size, block_size>>>(d_N, d_P, FILTER_RADIUS, width, height);
    }
    cudaDeviceSynchronize();

    // Stop recording time
    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);

    // Calculate elapsed time
    float elapsed_time;
    cudaEventElapsedTime(&elapsed_time, start, stop);

    // Print average time per kernel execution
    printf("Average time per convolution_2d_constant_mem_kernel: %.3f ms\n", elapsed_time / 10);

    // Copy output data back to host memory
    cudaMemcpy(P, d_P, width * height * sizeof(float), cudaMemcpyDeviceToHost);

    // verification
    verify_convolution_result(N, F, P, width, height, FILTER_RADIUS);

    // Print first 10 elements of output tensor
    for (int i = 0; i < 10; i++)
    {
        printf("%f, ", P[i]);
    }
    printf("\n");

    // Free device memory and destroy CUDA events
    cudaFree(d_N);
    cudaFree(d_P);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
}

void call_2d_tiled_constant_mem_kernel(float *N, float *F, float *P, int width, int height)
{
    // Device memory pointers
    float *d_N, *d_P;

    // Allocate device memory for input tensor
    cudaMalloc(&d_N, width * height * sizeof(float));
    // Allocate device memory for output tensor
    cudaMalloc(&d_P, width * height * sizeof(float));

    // Copy input tensor to device memory
    cudaMemcpy(d_N, N, width * height * sizeof(float), cudaMemcpyHostToDevice);
    // Copy filter tensor to constant memory
    cudaMemcpyToSymbol(dc_F, F, (2 * FILTER_RADIUS + 1) * (2 * FILTER_RADIUS + 1) * sizeof(float));

    // Block and grid size configuration
    dim3 block_size(IN_TILE_DIM, IN_TILE_DIM);
    dim3 grid_size((width + OUT_TILE_DIM - 1) / OUT_TILE_DIM,
                   (height + OUT_TILE_DIM - 1) / OUT_TILE_DIM);

    // Create CUDA events for timing
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // Warm-up execution to avoid first-time overhead
    convolution_2d_tiled_const_mem_kernel<<<grid_size, block_size>>>(d_N, d_P, width, height);
    cudaDeviceSynchronize();

    // Start recording time
    cudaEventRecord(start, 0);

    // Execute the kernel 10 times for averaging
    for (int i = 0; i < 10; i++)
    {
        convolution_2d_tiled_const_mem_kernel<<<grid_size, block_size>>>(d_N, d_P, width, height);
    }
    cudaDeviceSynchronize();

    // Stop recording time
    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);

    // Calculate elapsed time
    float elapsed_time;
    cudaEventElapsedTime(&elapsed_time, start, stop);

    // Print the average time per kernel execution
    printf("Average time per convolution_2d_tiled_const_mem_kernel: %.3f ms\n", elapsed_time / 10);

    // Copy output tensor to host memory
    cudaMemcpy(P, d_P, width * height * sizeof(float), cudaMemcpyDeviceToHost);

    // verification
    verify_convolution_result(N, F, P, width, height, FILTER_RADIUS);

    // Print the first 10 elements of the output tensor
    for (int i = 0; i < 10; i++)
    {
        printf("%f, ", P[i]);
    }
    printf("\n");

    // Free device memory
    cudaFree(d_N);
    cudaFree(d_P);

    // Destroy CUDA events
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
}

void call_2d_cached_tiled_constant_mem_kernel(float *N, float *F, float *P, int width, int height)
{
    // Device memory pointers
    float *d_N, *d_P;

    // Allocate device memory for input tensor
    cudaMalloc(&d_N, width * height * sizeof(float));
    // Allocate device memory for output tensor
    cudaMalloc(&d_P, width * height * sizeof(float));

    // Copy input tensor to device memory
    cudaMemcpy(d_N, N, width * height * sizeof(float), cudaMemcpyHostToDevice);
    // Copy filter tensor to constant memory
    cudaMemcpyToSymbol(dc_F, F, (2 * FILTER_RADIUS + 1) * (2 * FILTER_RADIUS + 1) * sizeof(float));

    // Configure block and grid size
    dim3 block_size(TILE_DIM, TILE_DIM);
    dim3 grid_size((width + TILE_DIM - 1) / TILE_DIM, (height + TILE_DIM - 1) / TILE_DIM);

    // Create CUDA events for timing
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // Warm-up execution (optional, avoids first-time overhead)
    convolution_2d_cached_tiled_const_mem_kernel<<<grid_size, block_size>>>(d_N, d_P, width, height);
    cudaDeviceSynchronize();

    // Start recording time
    cudaEventRecord(start, 0);

    // Execute the kernel 10 times for averaging
    for (int i = 0; i < 10; i++)
    {
        convolution_2d_cached_tiled_const_mem_kernel<<<grid_size, block_size>>>(d_N, d_P, width, height);
    }
    cudaDeviceSynchronize();

    // Stop recording time
    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);

    // Calculate elapsed time
    float elapsed_time;
    cudaEventElapsedTime(&elapsed_time, start, stop);

    // Print the average time per kernel execution
    printf("Average time per convolution_2d_cached_tiled_const_mem_kernel: %.3f ms\n", elapsed_time / 10);

    // Copy output tensor to host memory
    cudaMemcpy(P, d_P, width * height * sizeof(float), cudaMemcpyDeviceToHost);

    // verification
    verify_convolution_result(N, F, P, width, height, FILTER_RADIUS);

    // Print the first 10 elements of the output tensor
    for (int i = 0; i < 10; i++)
    {
        printf("%f, ", P[i]);
    }
    printf("\n");

    // Free device memory
    cudaFree(d_N);
    cudaFree(d_P);

    // Destroy CUDA events
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
}

int main(int argc, char **argv)
{

    // input tensor
    float *N;
    // filter tensor
    float *F;
    // output tensor
    float *P;

    // size of input tensor
    int width = 8192;
    int height = 8192;

    // allocate memory for input tensor
    N = (float *)malloc(width * height * sizeof(float));
    // allocate memory for filter tensor
    F = (float *)malloc((2 * FILTER_RADIUS + 1) * (2 * FILTER_RADIUS + 1) * sizeof(float));
    // allocate memory for output tensor
    P = (float *)malloc(width * height * sizeof(float));

    // initialize input tensor
    for (int i = 0; i < width * height; i++)
    {
        N[i] = i * 0.1f;
    }

    // initialize filter tensor
    for (int i = 0; i < (2 * FILTER_RADIUS + 1) * (2 * FILTER_RADIUS + 1); i++)
    {
        F[i] = i * 0.05f;
    }

    // call the basic kernel
    call_basic_kernel(N, F, P, width, height);

    // call the kernel with constant memory
    call_constant_mem_kernel(N, F, P, width, height);

    // call the tiled kernel
    call_2d_tiled_constant_mem_kernel(N, F, P, width, height);

    // call the cached tiled kernel
    call_2d_cached_tiled_constant_mem_kernel(N, F, P, width, height);

    // free memory
    free(N);
    free(F);
    free(P);

    return 0;
}

Average time per convolution_2d_basic_kernel: 12.987 ms
Verification Results:
Max error: 2.400000e+01
Average error: 1.313119e+00
Result differs more than the threshold!
7865.159668, 10241.650391, 12495.500977, 12496.775391, 12498.050781, 12499.325195, 12500.600586, 12501.875977, 12503.151367, 12504.425781, 
Average time per convolution_2d_constant_mem_kernel: 11.763 ms
Verification Results:
Max error: 2.400000e+01
Average error: 1.313119e+00
Result differs more than the threshold!
7865.159668, 10241.650391, 12495.500977, 12496.775391, 12498.050781, 12499.325195, 12500.600586, 12501.875977, 12503.151367, 12504.425781, 
Average time per convolution_2d_tiled_const_mem_kernel: 18.983 ms
Verification Results:
Max error: 2.400000e+01
Average error: 1.313119e+00
Result differs more than the threshold!
7865.159668, 10241.650391, 12495.500977, 12496.775391, 12498.050781, 12499.325195, 12500.600586, 12501.875977, 12503.151367, 12504.425781, 
Average time per convolution_2d_cached_tiled_const_me

### result

## Integer version

In [ ]:
%%cuda
#include <cuda.h>
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <math.h>

#define MASK_SIZE 5
#define FILTER_RADIUS (MASK_SIZE / 2)

// tile size
#define IN_TILE_DIM 32
#define OUT_TILE_DIM (IN_TILE_DIM - 2 * FILTER_RADIUS)

__constant__ int dc_F[(2 * FILTER_RADIUS + 1) * (2 * FILTER_RADIUS + 1)];

// Performing 2D convolution on the CPU to verify the correctness of the GPU results.
void host_convolution_2d(int *N, int *F, int *P, int width, int height, int r)
{
    int filter_dim = 2 * r + 1;
    for (int row = 0; row < height; row++)
    {
        for (int col = 0; col < width; col++)
        {
            int p_value = 0;
            for (int f_row = 0; f_row < filter_dim; f_row++)
            {
                for (int f_col = 0; f_col < filter_dim; f_col++)
                {
                    int n_row = row - r + f_row;
                    int n_col = col - r + f_col;
                    if (n_row >= 0 && n_row < height && n_col >= 0 && n_col < width)
                    {
                        p_value += N[n_row * width + n_col] * F[f_row * filter_dim + f_col];
                    }
                }
            }
            P[row * width + col] = p_value;
        }
    }
}

// Verify the difference between the GPU convolution result and the CPU result.
void verify_convolution_result(int *N, int *F, int *P_gpu, int width, int height, int r, int threshold = 0)
{
    // Allocate CPU output array
    int *P_cpu = (int *)malloc(width * height * sizeof(int));

    // Using CPU to perform convolution
    host_convolution_2d(N, F, P_cpu, width, height, r);

    // Calculation error
    int max_error = 0;
    long long sum_error = 0;
    for (int i = 0; i < width * height; i++)
    {
        int err = abs(P_cpu[i] - P_gpu[i]);
        if (err > max_error)
        {
            max_error = err;
        }
        sum_error += err;
    }

    double avg_error = (double)sum_error / (width * height);

    printf("Verification Results:\n");
    printf("Max error: %d\n", max_error);
    printf("Average error: %.6f\n", avg_error);

    if (max_error <= threshold)
    {
        printf("Result is correct within the threshold.\n");
    }
    else
    {
        printf("Result differs more than the threshold!\n");
    }

    // Release memory
    free(P_cpu);
}

// basic 2d convolution kernel
__global__ void convolution_2d_basic_kernel(int *N, int *F, int *P, int r, int width, int height)
{
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int row = blockIdx.y * blockDim.y + threadIdx.y;

    if (col < width && row < height) {
        int p_value = 0;
        int filter_dim = 2 * r + 1;
        for (int f_row = 0; f_row < filter_dim; f_row++)
        {
            for (int f_col = 0; f_col < filter_dim; f_col++)
            {
                int n_row = row - r + f_row;
                int n_col = col - r + f_col;
                if (n_row >= 0 && n_row < height && n_col >= 0 && n_col < width)
                {
                    p_value += N[n_row * width + n_col] * F[f_row * filter_dim + f_col];
                }
            }
        }
        P[row * width + col] = p_value;
    }
}

__global__ void convolution_2d_constant_mem_kernel(int *N, int *P, int r, int width, int height)
{
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int row = blockIdx.y * blockDim.y + threadIdx.y;

    if (col < width && row < height) {
        int p_value = 0;
        int filter_dim = 2 * r + 1;
        for (int f_row = 0; f_row < filter_dim; f_row++)
        {
            for (int f_col = 0; f_col < filter_dim; f_col++)
            {
                int n_row = row - r + f_row;
                int n_col = col - r + f_col;
                if (n_row >= 0 && n_row < height && n_col >= 0 && n_col < width)
                {
                    p_value += N[n_row * width + n_col] * dc_F[f_row * filter_dim + f_col];
                }
            }
        }
        P[row * width + col] = p_value;
    }
}

__global__ void convolution_2d_tiled_const_mem_kernel(int *N, int *P, int width, int height)
{
    __shared__ int s_N[IN_TILE_DIM][IN_TILE_DIM];

    int col = blockIdx.x * OUT_TILE_DIM + threadIdx.x - FILTER_RADIUS;
    int row = blockIdx.y * OUT_TILE_DIM + threadIdx.y - FILTER_RADIUS;

    if (col >= 0 && col < width && row >= 0 && row < height)
    {
        s_N[threadIdx.y][threadIdx.x] = N[row * width + col];
    }
    else
    {
        s_N[threadIdx.y][threadIdx.x] = 0;
    }

    __syncthreads();

    int tile_col = threadIdx.x - FILTER_RADIUS;
    int tile_row = threadIdx.y - FILTER_RADIUS;

    if (col >= 0 && col < width && row >= 0 && row < height)
    {
        if (tile_col >= 0 && tile_col < OUT_TILE_DIM && tile_row >= 0 && tile_row < OUT_TILE_DIM)
        {
            int p_value = 0;
            int filter_dim = 2 * FILTER_RADIUS + 1;
            for (int f_row = 0; f_row < filter_dim; f_row++)
            {
                for (int f_col = 0; f_col < filter_dim; f_col++)
                {
                    p_value += s_N[tile_row + f_row][tile_col + f_col] * dc_F[f_row * filter_dim + f_col];
                }
            }
            P[row * width + col] = p_value;
        }
    }
}

#define TILE_DIM 8
__global__ void convolution_2d_cached_tiled_const_mem_kernel(int *N, int *P, int width, int height)
{
    __shared__ int s_N[TILE_DIM][TILE_DIM];

    int col = blockIdx.x * TILE_DIM + threadIdx.x;
    int row = blockIdx.y * TILE_DIM + threadIdx.y;
    if (col >= 0 && col < width && row >= 0 && row < height)
    {
        s_N[threadIdx.y][threadIdx.x] = N[row * width + col];
    }
    else
    {
        s_N[threadIdx.y][threadIdx.x] = 0;
    }

    __syncthreads();

    if (col < width && row < height)
    {
        int p_value = 0;
        int filter_dim = 2 * FILTER_RADIUS + 1;
        for (int f_row = 0; f_row < filter_dim; f_row++)
        {
            for (int f_col = 0; f_col < filter_dim; f_col++)
            {
                int s_x = threadIdx.x - FILTER_RADIUS + f_col;
                int s_y = threadIdx.y - FILTER_RADIUS + f_row;

                if (s_x >= 0 && s_x < TILE_DIM && s_y >= 0 && s_y < TILE_DIM)
                {
                    p_value += s_N[s_y][s_x] * dc_F[f_row * filter_dim + f_col];
                }
                else
                {
                    int g_x = col - FILTER_RADIUS + f_col;
                    int g_y = row - FILTER_RADIUS + f_row;
                    if (g_y >= 0 && g_y < height && g_x >= 0 && g_x < width)
                    {
                        p_value += N[g_y * width + g_x] * dc_F[f_row * filter_dim + f_col];
                    }
                }
            }
        }
        P[row * width + col] = p_value;
    }
}

void call_basic_kernel(int *N, int *F, int *P, int width, int height)
{
    // Device memory pointers
    int *d_N, *d_F, *d_P;

    // Allocate device memory for input tensor
    cudaMalloc(&d_N, width * height * sizeof(int));
    // Allocate device memory for filter tensor
    cudaMalloc(&d_F, (2 * FILTER_RADIUS + 1) * (2 * FILTER_RADIUS + 1) * sizeof(int));
    // Allocate device memory for output tensor
    cudaMalloc(&d_P, width * height * sizeof(int));

    // Copy input tensor and filter to device memory
    cudaMemcpy(d_N, N, width * height * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_F, F, (2 * FILTER_RADIUS + 1) * (2 * FILTER_RADIUS + 1) * sizeof(int), cudaMemcpyHostToDevice);

    // Configure block and grid sizes
    dim3 block_size(32, 32);
    dim3 grid_size((width + block_size.x - 1) / block_size.x, (height + block_size.y - 1) / block_size.y);

    // Create CUDA events for timing
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    convolution_2d_basic_kernel<<<grid_size, block_size>>>(d_N, d_F, d_P, FILTER_RADIUS, width, height);
    cudaDeviceSynchronize();

    // Start recording time
    cudaEventRecord(start, 0);

    // Execute the kernel multiple times for benchmarking
    for (int i = 0; i < 10; i++)
    {
        convolution_2d_basic_kernel<<<grid_size, block_size>>>(d_N, d_F, d_P, FILTER_RADIUS, width, height);
    }
    cudaDeviceSynchronize();

    // Stop recording time
    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);

    // Calculate elapsed time
    float elapsed_time;
    cudaEventElapsedTime(&elapsed_time, start, stop);

    printf("Average time per convolution_2d_basic_kernel: %.3f ms\n", elapsed_time / 10);

    // Copy output tensor back to host memory
    cudaMemcpy(P, d_P, width * height * sizeof(int), cudaMemcpyDeviceToHost);

    // verification
    verify_convolution_result(N, F, P, width, height, FILTER_RADIUS);

    // Print the first 10 elements of the output tensor
    for (int i = 0; i < 10; i++)
    {
        printf("%d, ", P[i]);
    }
    printf("\n");

    // Free device memory
    cudaFree(d_N);
    cudaFree(d_F);
    cudaFree(d_P);

    // Destroy CUDA events
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
}

void call_constant_mem_kernel(int *N, int *F, int *P, int width, int height)
{
    int *d_N, *d_P;

    cudaMalloc(&d_N, width * height * sizeof(int));
    cudaMalloc(&d_P, width * height * sizeof(int));

    cudaMemcpy(d_N, N, width * height * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpyToSymbol(dc_F, F, (2 * FILTER_RADIUS + 1) * (2 * FILTER_RADIUS + 1) * sizeof(int));

    dim3 block_size(32, 32);
    dim3 grid_size((width + block_size.x - 1) / block_size.x,
                   (height + block_size.y - 1) / block_size.y);

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    convolution_2d_constant_mem_kernel<<<grid_size, block_size>>>(d_N, d_P, FILTER_RADIUS, width, height);
    cudaDeviceSynchronize();

    cudaEventRecord(start, 0);
    for (int i = 0; i < 10; i++)
    {
        convolution_2d_constant_mem_kernel<<<grid_size, block_size>>>(d_N, d_P, FILTER_RADIUS, width, height);
    }
    cudaDeviceSynchronize();

    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);

    float elapsed_time;
    cudaEventElapsedTime(&elapsed_time, start, stop);

    printf("Average time per convolution_2d_constant_mem_kernel: %.3f ms\n", elapsed_time / 10);

    cudaMemcpy(P, d_P, width * height * sizeof(int), cudaMemcpyDeviceToHost);

    verify_convolution_result(N, F, P, width, height, FILTER_RADIUS);

    for (int i = 0; i < 10; i++)
    {
        printf("%d, ", P[i]);
    }
    printf("\n");

    cudaFree(d_N);
    cudaFree(d_P);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
}

void call_2d_tiled_constant_mem_kernel(int *N, int *F, int *P, int width, int height)
{
    int *d_N, *d_P;

    cudaMalloc(&d_N, width * height * sizeof(int));
    cudaMalloc(&d_P, width * height * sizeof(int));

    cudaMemcpy(d_N, N, width * height * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpyToSymbol(dc_F, F, (2 * FILTER_RADIUS + 1) * (2 * FILTER_RADIUS + 1) * sizeof(int));

    dim3 block_size(IN_TILE_DIM, IN_TILE_DIM);
    dim3 grid_size((width + OUT_TILE_DIM - 1) / OUT_TILE_DIM,
                   (height + OUT_TILE_DIM - 1) / OUT_TILE_DIM);

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    convolution_2d_tiled_const_mem_kernel<<<grid_size, block_size>>>(d_N, d_P, width, height);
    cudaDeviceSynchronize();

    cudaEventRecord(start, 0);
    for (int i = 0; i < 10; i++)
    {
        convolution_2d_tiled_const_mem_kernel<<<grid_size, block_size>>>(d_N, d_P, width, height);
    }
    cudaDeviceSynchronize();

    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);

    float elapsed_time;
    cudaEventElapsedTime(&elapsed_time, start, stop);

    printf("Average time per convolution_2d_tiled_const_mem_kernel: %.3f ms\n", elapsed_time / 10);

    cudaMemcpy(P, d_P, width * height * sizeof(int), cudaMemcpyDeviceToHost);

    verify_convolution_result(N, F, P, width, height, FILTER_RADIUS);

    for (int i = 0; i < 10; i++)
    {
        printf("%d, ", P[i]);
    }
    printf("\n");

    cudaFree(d_N);
    cudaFree(d_P);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
}

void call_2d_cached_tiled_constant_mem_kernel(int *N, int *F, int *P, int width, int height)
{
    int *d_N, *d_P;

    cudaMalloc(&d_N, width * height * sizeof(int));
    cudaMalloc(&d_P, width * height * sizeof(int));

    cudaMemcpy(d_N, N, width * height * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpyToSymbol(dc_F, F, (2 * FILTER_RADIUS + 1) * (2 * FILTER_RADIUS + 1) * sizeof(int));

    dim3 block_size(TILE_DIM, TILE_DIM);
    dim3 grid_size((width + TILE_DIM - 1) / TILE_DIM, (height + TILE_DIM - 1) / TILE_DIM);

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    convolution_2d_cached_tiled_const_mem_kernel<<<grid_size, block_size>>>(d_N, d_P, width, height);
    cudaDeviceSynchronize();

    cudaEventRecord(start, 0);
    for (int i = 0; i < 10; i++)
    {
        convolution_2d_cached_tiled_const_mem_kernel<<<grid_size, block_size>>>(d_N, d_P, width, height);
    }
    cudaDeviceSynchronize();

    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);

    float elapsed_time;
    cudaEventElapsedTime(&elapsed_time, start, stop);

    printf("Average time per convolution_2d_cached_tiled_const_mem_kernel: %.3f ms\n", elapsed_time / 10);

    cudaMemcpy(P, d_P, width * height * sizeof(int), cudaMemcpyDeviceToHost);

    verify_convolution_result(N, F, P, width, height, FILTER_RADIUS);

    for (int i = 0; i < 10; i++)
    {
        printf("%d, ", P[i]);
    }
    printf("\n");

    cudaFree(d_N);
    cudaFree(d_P);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
}

int main(int argc, char **argv)
{
    int width = 8192;
    int height = 8192;

    int *N = (int *)malloc(width * height * sizeof(int));
    int *F = (int *)malloc((2 * FILTER_RADIUS + 1) * (2 * FILTER_RADIUS + 1) * sizeof(int));
    int *P = (int *)malloc(width * height * sizeof(int));

    for (int i = 0; i < width * height; i++)
    {
        N[i] = i;
    }

    for (int i = 0; i < (2 * FILTER_RADIUS + 1) * (2 * FILTER_RADIUS + 1); i++)
    {
        F[i] = i;
    }

    call_basic_kernel(N, F, P, width, height);
    call_constant_mem_kernel(N, F, P, width, height);
    call_2d_tiled_constant_mem_kernel(N, F, P, width, height);
    call_2d_cached_tiled_constant_mem_kernel(N, F, P, width, height);

    free(N);
    free(F);
    free(P);

    return 0;
}

Average time per convolution_2d_basic_kernel: 12.549 ms
Verification Results:
Max error: 0
Average error: 0.000000
Result is correct within the threshold.
1573032, 2048330, 2499100, 2499355, 2499610, 2499865, 2500120, 2500375, 2500630, 2500885, 
Average time per convolution_2d_constant_mem_kernel: 10.876 ms
Verification Results:
Max error: 0
Average error: 0.000000
Result is correct within the threshold.
1573032, 2048330, 2499100, 2499355, 2499610, 2499865, 2500120, 2500375, 2500630, 2500885, 
Average time per convolution_2d_tiled_const_mem_kernel: 8.806 ms
Verification Results:
Max error: 0
Average error: 0.000000
Result is correct within the threshold.
1573032, 2048330, 2499100, 2499355, 2499610, 2499865, 2500120, 2500375, 2500630, 2500885, 
Average time per convolution_2d_cached_tiled_const_mem_kernel: 20.389 ms
Verification Results:
Max error: 0
Average error: 0.000000
Result is correct within the threshold.
1573032, 2048330, 2499100, 2499355, 2499610, 2499865, 2500120, 2500375, 25

### result